In [1]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)
								 

Using TensorFlow backend.


In [2]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}


In [3]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output

    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling
	

In [4]:
dream = model.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x
	

In [48]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x


In [49]:
import numpy as np

''' Playing with these hyperparameters will let you achieve new effects. '''
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = 10.

base_image_path='C:/Users/sy3445/Desktop/test/pic olive/olive.jpg'
img = preprocess_image(base_image_path)
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape=tuple([int(dim/(octave_scale**i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])
for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img,shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img,shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail=same_size_original-upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = resize_img(original_img,shape)
    save_img(img, fname='dream_at_scale_'+str(shape)+'.jpg')
#     save_img(img,fname='dream_at_scale_' + str(shape) + '.png')
save_img(img, fname='final_dream.png')
 

C:\Users\sy3445\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:600: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Processing image shape (221, 333)
...Loss value at 0 : 2.26574
...Loss value at 1 : 2.53316
...Loss value at 2 : 3.20042
...Loss value at 3 : 3.85101
...Loss value at 4 : 4.46587
...Loss value at 5 : 5.03753
...Loss value at 6 : 5.64588
...Loss value at 7 : 6.18758
...Loss value at 8 : 6.71826
...Loss value at 9 : 7.21225
...Loss value at 10 : 7.7378
...Loss value at 11 : 8.29152
...Loss value at 12 : 8.74453
...Loss value at 13 : 9.17139
...Loss value at 14 : 9.61789
Processing image shape (310, 466)
...Loss value at 0 : 3.23584
...Loss value at 1 : 4.59226
...Loss value at 2 : 5.68262
...Loss value at 3 : 6.59845
...Loss value at 4 : 7.43762
...Loss value at 5 : 8.2492
...Loss value at 6 : 8.97983
...Loss value at 7 : 9.58689
Processing image shape (434, 653)
...Loss value at 0 : 3.34489
...Loss value at 1 : 4.53527
...Loss value at 2 : 5.62225
...Loss value at 3 : 6.59756
...Loss value at 4 : 7.50404
...Loss value at 5 : 8.37035
...Loss value at 6 : 9.14865
...Loss value at 7 : 9.91